In [52]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os

def get_cuboid_mesh(verts, elems, values, cmap="jet"):
    
    tri_idx = np.array([[0,0,0,0,0,0,6,6,6,6,6,6],
                        [2,3,7,4,1,5,2,3,1,5,4,7],
                        [1,2,3,7,5,4,3,7,2,1,5,4]], dtype=int)
    mesh = o3d.geometry.TriangleMesh()
    tris = np.zeros([12*elems.shape[0],3], dtype=int)
    for i, elem in enumerate(elems):
        tris[i*12:(i+1)*12,:] = elem[tri_idx].T-1
    
    mesh.vertices = o3d.utility.Vector3dVector(verts)
    mesh.triangles = o3d.utility.Vector3iVector(tris)

    cmap = plt.get_cmap(cmap)
    lb, ub = min(values), max(values)
    values = (values - lb) / (ub - lb)
    mesh.vertex_colors = o3d.utility.Vector3dVector(cmap(values)[:,:3])
    return mesh
                          
data_path = r"C:\Users\XuanLiang\Documents\GitHub\netfabb\dataset\\"
data_files = os.listdir(data_path)

In [59]:
i = 0
data = np.load(data_path + data_files[i])
verts, elems, disp = data["verts"], data["elems"], data["disp"]
mesh = get_cuboid_mesh(verts, elems, disp[:,2])
o3d.visualization.draw_geometries([mesh])

In [54]:
import numpy as np
import struct

def read80(f):
    return f.read(80).decode('utf-8').strip()

def read_floats(f,N):
    arr = np.array(struct.unpack(f"<{N}f", f.read(4*N)))
    return arr[0] if N == 1 else arr

def read_ints(f,N):
    arr = np.array(struct.unpack(f"<{N}i", f.read(4*N)))
    return arr[0] if N == 1 else arr

def read_geo_binary(path):
    with open(path, 'rb') as f:

        ############ Info
        assert read80(f) == 'Fortran Binary'
        description1 = read80(f)
        description2 = read80(f)
        assert read80(f) == 'node id off'
        assert read80(f) == 'element id off'
        # extents_str = read80(f)       # No extents
        # extents = read_floats(f,6)    # No extents

        ############ Begin part 1
        assert read80(f) == 'part'
        assert read_ints(f,1) == 1 # Should be 1 part only
        description3 = read80(f)

        ############ Coordinates
        assert read80(f) == 'coordinates'
        nn = read_ints(f,1)
        # node_ids = read_ints(f,nn) # node id is off
        x = read_floats(f,nn)
        y = read_floats(f,nn)
        z = read_floats(f,nn)
        nodes = np.vstack([x,y,z]).T

        ############ Elements
        element_type = read80(f)
        assert(element_type == 'hexa8')
        # element_ids = read_ints(f,nn) # element id is off
        ne = read_ints(f, 1)
        elems = read_ints(f, 8*ne).reshape(ne,8)

    data = dict(description1=description1, description2=description2, description3=description3, 
                nn=nn, nodes=nodes, element_type=element_type, ne=ne, elems=elems)
    return data

def read_ens_binary(path, num_nodes, num_values):
    with open(path, 'rb') as f:
        description = read80(f)
        assert(read80(f) == 'part')
        assert(read_ints(f,1) == 1)
        assert(read80(f) == 'coordinates')
        arr = read_floats(f,num_nodes*num_values)
    data = arr.reshape(num_values, num_nodes).T
    return dict(description=description, data=data)

In [47]:
data = read_geo_binary("example.geo")
disp_data = read_ens_binary("example.ens", data["nn"], 3)
verts, elems, disp = data["nodes"], data["elems"], disp_data["data"]
mesh = get_cuboid_mesh(verts, elems, disp[:,2])
o3d.visualization.draw_geometries([mesh])

FileNotFoundError: [Errno 2] No such file or directory: 'example.geo'